# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [14]:
# imports
import os
import requests
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display, update_display

In [15]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [16]:
# set up environment
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

openai = OpenAI()

In [25]:
# here is the question; type over this to ask something new
question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

messages = [
    {
        "role": "system",
        "content": "You are an assistant that will answer technical questions for programming. When giving try to give in an explanation and and example. \
        Explaining the parts of the example if any. Respond in markdown format. If in your response you see the word 'dict', 'and', or 'books', I want you to underline the word. \
        At the very end of the response, create a witty joke about the number of rockets you found!"
    },
    {
        "role": "user",
        "content": question
    }
]


In [26]:
# Get gpt-4o-mini to answer, with streaming

def stream_response(stream):
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

def call_open_ai_stream(prompt):
    stream = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=prompt,
        stream=True
    )
    
    stream_response(stream)


call_open_ai_stream(messages)

The provided code snippet is using the `yield from` syntax in conjunction with a set comprehension to produce a generator.

### Explanation

1. **Set Comprehension**: 
   python
   {book.get("author") for book in books if book.get("author")}
   
   - This part of the code creates a set by iterating over each `book` in the `books` collection.
   - For each `book`, it uses `.get("author")` to retrieve the value associated with the key `"author"`.
   - The `if book.get("author")` part ensures that only books that have an author (i.e., where `book.get("author")` returns a truthy value) are considered for the set. If a book does not have an author, it will be excluded from the set.

2. **`yield from`**:
   - The `yield from` statement is used in a generator function to yield values from another iterable or generator. In this case, it allows the generator to produce each author in the set one by one.
   - So, the entire line allows a generator to yield each unique author from the collection of `books`.

### Example

Here’s a more complete example:

python
def get_authors(books):
    yield from {book.get("author") for book in books if book.get("author")}

# Example usage
books = [
    {"title": "Book 1", "author": "Author A"},
    {"title": "Book 2", "author": "Author B"},
    {"title": "Book 3"},
    {"title": "Book 4", "author": "Author A"},
]

for author in get_authors(books):
    print(author)


### Breakdown of the Example:

- **Defining a Function**: `get_authors(books)` is a generator function that will yield authors from the provided list of `books`.
- **Input List**: The `books` list includes dictionaries representing books, with most having an `"author"` key.
- **Using the Function**: When we loop through the authors produced by `get_authors(books)`, it will print unique authors:
  
  Author A
  Author B
  
  Notice how it yields `Author A` only once, even though two books share that author.

In summary, this code efficiently compiles a list of unique authors from a potentially large collection of books, while also ensuring that any entries without an author are excluded.

And as for the rockets, I found a lot of them, probably enough to launch a “space book” club! 🚀

In [21]:
# Get Llama 3.2 to answer

def call_ollama_stream(prompt):
    ollama = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
    stream = ollama.chat.completions.create(
        model=MODEL_LLAMA,
        messages=prompt,
        stream=True
    )

    stream_response(stream)

call_ollama_stream(messages)
    

**Unpacking a Nested Generator**
=====================================

The provided code snippet is using generator expression and `yield from` to extract the authors of books from a list. Here's a breakdown:

python
yield from {book.get("author") for book in books if book.get("author")}


*   `{...}` : This syntax creates an empty dictionary comprehension, which might seem unusual at first. However, it serves a purpose similar to `dict comprehension`.

However there is not enough information about the code structure and libraries used. So Below I'll give 2 possible structure.

### Structure 1

python
import collections.abc

def get_books(books):
    for book in books:
        yield from {
            key: value 
            for key, value in book.items()
            if key != 'author' or (key == 'author' and value)
        }

def find_authors():
    authors = {}
    for book in books:
        author = next(book.values())
        if not author:
            continue
        if author not in authors.values():
            yield author 

# Example usage
books = [
    {"title": "Book1", "author": "John"},
    {"title": "Book2", "author": ""},
    {"title": "-3-Book3", "author": "Alessandro"}
]
books = [book for book in books if book] # drop the empty books

authors = set(list(find_authors()))
print(authors) 

In this code `get_books` is function that yield each line of a JSON. We are also filtering out some lines (as requested).

python
    def find_authors():
        authors = {}
        for book in books:
            author = next(book.values())
            if not author:
                continue
            if author not in authors.values():
                yield author 

`find_authors()` function filters duplicate value and also skipping on None.